In [27]:
import numpy as np
import MFCC
import scipy
from scipy.io.wavfile import read
import fnmatch
import os
import glob
import pickle

# Problem 1

In [4]:
def sample_gmmhmm(gmmhmm, n_sim):
    """
    Simulate sampling from a GMMHMM.
    gmmhmm = [A, weights, means, covars, pi]
    Returns
    -------
    states : ndarray of shape (n_sim,)
    The sequence of states
    obs : ndarray of shape (n_sim, K)
    The generated observations (column vectors of length K)
    """
    A,weights,means,covars,pi = gmmhmm
    states = np.empty(n_sim,dtype='int32')
    obs = np.empty((n_sim,means.shape[1]))
    states[0] = int(np.argmax(np.random.multinomial(1,pi)))
    for i in xrange(n_sim):
        sample_component = int(np.argmax(
                np.random.multinomial(1, weights[states[i],:])))
        obs[i] = np.random.multivariate_normal(
                        means[states[i], sample_component, :], 
                        covars[states[i], sample_component, :, :])
        if i+1 != n_sim:
            states[i+1] = int(np.argmax(
                np.random.multinomial(1,A[states[i]])))
    return states,obs

Testing problem 1

In [5]:
A = np.array([[.65, .35], [.15, .85]])
pi = np.array([.8, .2])
weights = np.array([[.7, .2, .1], [.1, .5, .4]])
means1 = np.array([[0., 17., -4.], [5., -12., -8.], [-16., 22., 2.]])
means2 = np.array([[-5., 3., 23.], [-12., -2., 14.], [15., -32., 0.]])
means = np.array([means1, means2])
covars1 = np.array([5*np.eye(3), 7*np.eye(3), np.eye(3)])
covars2 = np.array([10*np.eye(3), 3*np.eye(3), 4*np.eye(3)])
covars = np.array([covars1, covars2])
gmmhmm = [A, weights, means, covars, pi]
sample_gmmhmm(gmmhmm,10)

(array([0, 1, 1, 1, 1, 0, 0, 0, 1, 1], dtype=int32),
 array([[  6.33912087, -11.61318906,  -8.41196492],
        [ 18.46593289, -33.75325316,  -0.30483888],
        [-13.03661446,  -0.58311584,  13.19631393],
        [-13.53074848,  -1.14335225,  12.8938018 ],
        [-10.98557849,  -1.0775124 ,  16.03938241],
        [  9.32049172, -13.16615285,  -4.93089293],
        [ -2.91514145,  16.86142171,  -1.74817317],
        [  4.90953084, -13.80801675,  -8.64075437],
        [ 13.3965406 , -33.23650346,  -4.59064789],
        [-11.63321304,  -3.5113175 ,  12.90834922]]))

# Problem 2

In [33]:
names = ["Biology","Mathematics","PoliticalScience",
         "Psychology","Statistics",]
mfccs = [[] for i in range(len(names))]
for i,name in enumerate(names):
    for file in glob.glob("wavs/{} (*).wav".format(name)):
        mfccs[i].append(MFCC.extract(read(file)[1]))

# Problem 3

In [37]:
samples = [[mfccs[i][j] for j in range(20)] 
                       for i in range(len(names))]
tests = [[mfccs[i][j] for j in range(20,30)] 
                       for i in range(len(names))]

In [35]:
def initialize(N):
    pi = np.random.rand(N)
    pi /= pi.sum()
    A = np.random.rand(N,N)
    for i in range(N):
        A[i] /= A[i].sum()
    return pi,A

In [ ]:
import gmmhmm

models = []
for i in range(len(samples)):
    best_log = -np.infty
    best_model = None
    for j in xrange(10):
        startprob, transmat = initialize(5)
        model = gmmhmm.GMMHMM(n_components=5, n_mix=3, 
            transmat=transmat, startprob=startprob, cvtype='diag')
        # these values for covars_prior and var should work well for this problem
        model.covars_prior = 0.01
        model.fit(samples[i], init_params='mc', var=0.1)
        if model.logprob > best_log:
            best_log = model.logprob
            best_model = model
    models.append(best_model)
pickle.dump(models,open("models.pickle","wb"))

Test how well it did

In [75]:
scores = np.empty((len(tests),10,len(models)))
bests = np.empty((len(tests),10))
for i in xrange(len(tests)):
    for j in range(10):
        for k in xrange(len(models)):
            scores[i,j,k] = models[k].score(tests[i][j])
        bests[i,j] = np.argmax(scores[i,j,:])


accuracies = np.empty(len(names))
for i in range(len(names)):
    accuracies[i] = np.sum(bests[i] == i)/10.
d = {n:a for (n,a) in zip(names,accuracies)}
print bests
print d

[[ 0.  3.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.]
 [ 3.  3.  3.  3.  3.  3.  3.  3.  3.  3.]
 [ 4.  4.  4.  4.  4.  4.  4.  4.  4.  4.]]
{'Mathematics': 1.0, 'Biology': 0.80000000000000004, 'Statistics': 1.0, 'Psychology': 1.0, 'PoliticalScience': 1.0}


Biology had bad accuracy, everything else will be good